In [29]:
# Takes in csv of original, raw tweets and outputs hashtag edge lists for use in making networks

In [30]:
import pandas as pd
import numpy as np
import string
import re
from cleaners import clean_hashtags, hashtags_to_df, load_tweets, get_hashtag_edges
from cleaners import consolidate_edges
import itertools
from itertools import combinations
from sklearn import preprocessing

In [31]:
df = load_tweets('data/stop_steal.csv')

In [32]:
hashtag_df = hashtags_to_df(df.hashtags)

In [33]:
hashtag_df = hashtag_df.fillna(value=np.nan)

In [34]:
edges_df = get_hashtag_edges(hashtag_df)

In [35]:
# Dropping nan values in first two rows
edges_df.drop(edges_df.index[0:2], inplace=True)

In [36]:
#(edges_df['edges'].value_counts() > 100) && (edges_df['edges'].value_counts() < 2000) # this goes in brackets above

In [37]:
# selects all edges with value counts greater than n
n = 45
top_edges = edges_df['edges'].value_counts() [edges_df['edges'].value_counts() > n].reset_index()
top_edges.rename(columns={'index' : 'edges', 'edges' : 'weight'}, inplace=True)

In [38]:
# consolidates duplicates (with reverse order), adds up weights, puts into new dataframe
new_top_edges = consolidate_edges(top_edges)

In [39]:
# normalize weights
x = new_top_edges[['weight']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized = pd.DataFrame(x_scaled, columns=['weights'])
norm_weights=normalized['weights']

new_top_edges.insert(1, "weights", norm_weights, True)
new_top_edges.drop(columns=['weight'], inplace=True)

In [40]:
new_top_edges.to_csv('data/hash_edges_45.csv')